In [ ]:
import torch.nn as nn
from torchvision.models import resnet50, ResNet50_Weights
model=resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
model=resnet50(weights="IMAGENET1K_V1")
model=resnet50(pretrained=True)  # deprecated
model=resnet50(True)  # deprecated

num_filters = model.fc.in_features

model.fc = nn.Linear(num_filters, 10)

In [ ]:
for param in model.parameters():
    param.requires_grad = False
    
if isinstance(model, torchvision.models.ResNet):
    layers_to_unfreeze = list(model.children())[-5:]
    for layer in layers_to_unfreeze:
        for param in layer.parameters():
            param.requires_grad = True